In [5]:
import pandas as pd
import glob

In [12]:
for _ in glob.glob('New*labelled*.csv'):
    temp = pd.read_csv(_)
    filterTemp = temp[~temp['Sample ID'].isna()]
    filterTemp.reset_index().iloc[:,2:].to_csv('DropRedundanced_'+_)

In [64]:
enhance = pd.read_csv('DropRedundanced_New_Enhance_Translocation(Outlier)_600_labelled_1196.csv')
enhance['Phenotype'] = ['Enhance']*enhance.shape[0]
inhibit = pd.read_csv('DropRedundanced_New_Inhibit_Translocation(Outlier)_1500_labelled_3006.csv')
inhibit['Phenotype'] = ['Inhibit']*inhibit.shape[0]
trans = pd.concat([inhibit,enhance],axis=0)
trans['Translocation(Outlier)_Level']=inhibit['Translocation(Outlier)_Level_1500'].tolist()+enhance['Translocation(Outlier)_Level_600'].tolist()
trans = trans.drop(columns=['Translocation(Outlier)_Level_1500','Translocation(Outlier)_Level_600']).iloc[:,1:]

In [65]:
ess_path = r'/media/benz/Sandisk_SSD/After_proposal/Essential Data/'
model = pd.read_csv(ess_path+'Adaptive_analysis_result_Trans_ssmd_name_May_2021.csv')
model = model[['Gene_Symbol','Scramble_mean_real','Scramble_Trans (Model)','ssmd_Trans']]
model.columns = ['Gene_Symbol','Scramble_Nuc/Inf_Local','Scramble_Nuc/Inf_Model','ssmd_Trans']
model

,Gene_Symbol,Scramble_Nuc/Inf_Local,Scramble_Nuc/Inf_Model,ssmd_Trans
0,A1BG,2.2930,1.917899,0.7154
1,A1CF,2.2740,1.976380,0.2679
2,A2M,1.8129,1.636779,0.5385
3,A2ML1,1.5086,1.226672,0.2037
4,A3GALT2,1.7343,1.582218,-0.5494
...,...,...,...,...
18345,ZYX,1.6432,1.151042,0.2134
18346,ZZEF1,2.5361,2.579460,0.1337
18347,ZZZ3,2.3863,2.103415,-0.4087
18348,hCG_1729902,1.6355,1.217976,-1.1987


In [66]:
tempSelect = pd.DataFrame()
for g in trans['Gene']:
    select = model.set_index('Gene_Symbol').loc[[g],['Scramble_Nuc/Inf_Local','Scramble_Nuc/Inf_Model','ssmd_Trans']]
    tempSelect = tempSelect.append(select)

In [67]:
tempTrans = trans.set_index('Gene')
trans = pd.concat([tempTrans,tempSelect],axis=1).reset_index()
trans

,index,Total_cells,Inf_cells,Uninf_cells,Bac/Inf,Nuc/Inf,Cytomem/Inf,Infected/Total_cells,Nuc/IntraBac,Inf/Uninf_cells,...,Full Gene Name,384-well Location,Plate,Batch,New_name,Phenotype,Translocation(Outlier)_Level,Scramble_Nuc/Inf_Local,Scramble_Nuc/Inf_Model,ssmd_Trans
0,ABHD4,233.666667,195.333333,38.333333,4.173101,2.511936,1.606565,0.853867,0.602833,6.498433,...,abhydrolase domain containing 4,G4,Plate-23,Batch-05,ABHD4-G4,Inhibit,Adap_Level_1500,2.3836,2.988717,-4.0410
1,AKIRIN1,424.000000,300.333333,123.666667,3.738660,2.411676,1.266250,0.710000,0.645167,2.473600,...,akirin 1,C5,Plate-46,Batch-08,AKIRIN1-C5,Inhibit,Adap_Level_1500,3.4852,2.709135,-4.4827
2,AKT2,358.666667,230.000000,128.666667,2.601271,1.852009,0.711823,0.643233,0.712667,1.842967,...,v-akt murine thymoma viral oncogene homolog 2,G6,Plate-01,Batch-01,AKT2-G6,Inhibit,Adap_Level_1500,1.8129,2.012564,-1.6762
3,AMMECR1L,145.666667,114.666667,31.000000,3.942848,1.820201,2.112958,0.795733,0.464300,4.679533,...,AMMECR1-like,I10,Plate-47,Batch-08,AMMECR1L-I10,Inhibit,Adap_Level_1500,2.2702,2.332110,-3.7771
4,ARID5A,328.666667,258.333333,70.333333,3.869417,2.608006,1.195872,0.783767,0.675400,3.900633,...,AT rich interactive domain 5A (MRF1-like),O16,Plate-31,Batch-06,ARID5A-O16,Inhibit,Adap_Level_1500,2.5361,2.801746,-4.4057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4197,CCDC67,219.000000,170.666667,48.333333,3.102056,1.501473,1.541456,0.793433,0.486933,4.286400,...,coiled-coil domain containing 67,B6,Plate-52,Batch-08,CCDC67-B6,Enhance,Con_Level_600,1.2798,1.889764,-2.0244
4198,C17orf56,248.000000,165.666667,82.333333,2.880768,1.508839,1.371168,0.683733,0.535600,2.216400,...,chromosome 17 open reading frame 56,A5,Plate-52,Batch-08,C17orf56-A5,Enhance,Con_Level_600,1.2798,1.705351,-0.6343
4199,TMEM9,230.666667,184.666667,46.000000,7.859411,4.730418,3.055591,0.801000,0.603167,4.075167,...,transmembrane protein 9,B21,Plate-54,Batch-09,TMEM9-B21,Enhance,Con_Level_600,4.4324,4.962262,-0.1920
4200,STRA8,75.333333,59.333333,16.000000,4.732763,2.658960,1.981639,0.805733,0.579867,7.155033,...,stimulated by retinoic acid 8,L6,Plate-56,Batch-09,STRA8-L6,Enhance,Con_Level_600,2.2034,2.770325,-0.1369


In [90]:
meanScramble = pd.read_csv(ess_path+'Mean_Scramble_allPlate.csv').iloc[:,1:]
meanScramble['Plate'] = meanScramble['Plate'].str.replace('_P','-')
meanScramble

,Batch,Plate,Scramble_Total_cells_mean,Scramble_Inf_cells_mean,Scramble_UnInf_cells_mean,Scramble_Bac/Inf_mean,Scramble_Nuc/Inf_mean,Scramble_Cytomem/Inf_mean,Scramble_Infected/Total_cells_mean,Scramble_Nuc/IntraBac_mean,Scramble_Inf/Uninf_cells_mean
0,Batch_01,Plate-01,389.833333,225.944444,163.888889,2.360099,1.812933,0.513102,0.590450,0.767694,1.549933
1,Batch_01,Plate-02,422.588235,193.647059,228.941176,1.941390,1.316233,0.589098,0.479341,0.671318,1.058329
2,Batch_01,Plate-03,454.062500,215.125000,238.937500,2.124545,1.479357,0.589548,0.513313,0.697338,1.128737
3,Batch_01,Plate-04,456.352941,190.176471,266.176471,1.890606,1.261447,0.593029,0.443935,0.665600,0.843706
4,Batch_02,Plate-05,415.777778,240.000000,175.777778,3.304939,2.412302,0.851695,0.653883,0.737211,2.343494
...,...,...,...,...,...,...,...,...,...,...,...
56,Batch_09,Plate-57,269.764706,187.941176,81.823529,3.140230,2.011550,1.098060,0.714241,0.645659,2.798841
57,Batch_09,Plate-58,292.411765,198.000000,94.411765,2.985654,1.862057,1.090792,0.702259,0.625759,2.603582
58,Batch_09,Plate-59,298.117647,195.588235,102.529412,3.074128,1.919303,1.123615,0.704629,0.625800,2.847824
59,Batch_09,Plate-60,244.055556,162.888889,81.166667,2.817310,1.635502,1.146312,0.686544,0.580333,2.402700


In [92]:
tempSelect = pd.DataFrame()
for p in trans['Plate'].tolist():
    select = meanScramble.set_index(['Plate']).loc[[p],:]
    tempSelect = tempSelect.append(select)

tempTrans = trans.set_index('Plate')
trans = pd.concat([tempTrans,tempSelect],axis=1).reset_index()
trans['LevelShort'] = [_.split('_')[-1] for _ in trans['Level']]
trans

,Plate,index,Total_cells,Inf_cells,Uninf_cells,Bac/Inf,Nuc/Inf,Cytomem/Inf,Infected/Total_cells,Nuc/IntraBac,...,Batch,Scramble_Total_cells_mean,Scramble_Inf_cells_mean,Scramble_UnInf_cells_mean,Scramble_Bac/Inf_mean,Scramble_Nuc/Inf_mean,Scramble_Cytomem/Inf_mean,Scramble_Infected/Total_cells_mean,Scramble_Nuc/IntraBac_mean,Scramble_Inf/Uninf_cells_mean
0,Plate-23,ABHD4,233.666667,195.333333,38.333333,4.173101,2.511936,1.606565,0.853867,0.602833,...,Batch_05,349.333333,256.777778,92.555556,3.182273,2.383568,0.756675,0.749683,0.749789,3.194667
1,Plate-46,AKIRIN1,424.000000,300.333333,123.666667,3.738660,2.411676,1.266250,0.710000,0.645167,...,Batch_08,295.866667,219.466667,76.400000,5.480401,3.485154,1.891658,0.757807,0.636807,3.581920
2,Plate-01,AKT2,358.666667,230.000000,128.666667,2.601271,1.852009,0.711823,0.643233,0.712667,...,Batch_01,389.833333,225.944444,163.888889,2.360099,1.812933,0.513102,0.590450,0.767694,1.549933
3,Plate-47,AMMECR1L,145.666667,114.666667,31.000000,3.942848,1.820201,2.112958,0.795733,0.464300,...,Batch_08,293.055556,208.000000,85.055556,3.558111,2.270168,1.239771,0.725022,0.640239,2.929594
4,Plate-31,ARID5A,328.666667,258.333333,70.333333,3.869417,2.608006,1.195872,0.783767,0.675400,...,Batch_06,318.750000,239.000000,79.750000,3.468442,2.536126,0.896735,0.756256,0.732744,3.346025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4197,Plate-52,CCDC67,219.000000,170.666667,48.333333,3.102056,1.501473,1.541456,0.793433,0.486933,...,Batch_08,310.647059,181.294118,129.352941,2.364958,1.279763,1.059640,0.595924,0.546665,1.540476
4198,Plate-52,C17orf56,248.000000,165.666667,82.333333,2.880768,1.508839,1.371168,0.683733,0.535600,...,Batch_08,310.647059,181.294118,129.352941,2.364958,1.279763,1.059640,0.595924,0.546665,1.540476
4199,Plate-54,TMEM9,230.666667,184.666667,46.000000,7.859411,4.730418,3.055591,0.801000,0.603167,...,Batch_09,231.166667,182.916667,48.250000,7.185215,4.432358,2.600953,0.792700,0.616350,4.056700
4200,Plate-56,STRA8,75.333333,59.333333,16.000000,4.732763,2.658960,1.981639,0.805733,0.579867,...,Batch_09,257.588235,186.588235,71.000000,3.436448,2.203368,1.188805,0.740100,0.645918,3.074124


In [95]:
trans.to_csv('Test_trans.csv') # After exported data,i change order of columns and send it to team